In [1]:
# Use server from examples/servers/streamable-http-stateless/
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_mcp_adapters.client import MultiServerMCPClient

load_dotenv()

True

In [11]:
# pyright: strict = false
# type: ignore
async def call_todo_agent(config: RunnableConfig, memory: MemorySaver) -> None:
    """Run a class todo agent using the react agent framework and a remote MCP server."""
    client = MultiServerMCPClient({
        "fastapi-mcp": {
            "url": "http://localhost:8000/mcp",
            "transport": "sse",
        }
    })
    remote_tools = await client.get_tools()
    graph = create_react_agent(
        "groq:llama-3.3-70b-versatile", tools=remote_tools, checkpointer=memory)

    # Create event stream
    events = graph.astream_events({
        "messages": [
            HumanMessage(
                content="show me my completed todos",
            ),
        ]
    }, config=config, version="v2")

    # Handle stream output
    async for event in events:
        if event["event"] == "on_chat_model_stream":
            if "chunk" in event["data"]:
                print(event["data"]["chunk"].content, end="", flush=True)

In [12]:
shared_memory = MemorySaver()
 # Define thread config
thread_config = RunnableConfig(
    {"configurable": {
        "thread_id": 19,
    }}
)

await call_todo_agent(config=thread_config, memory=shared_memory)